# **PRÁCTICA 2 · PARTE 2: RECUPERACIÓN DE INFORMACIÓN**

__GRUPO 7__

Beatriz Herguedas Pinedo

Pablo Hernández Aguado

\* En la entrega previa se había subido una versión antigua de este cuaderno.

#### Bibliotecas y funciones importadas.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_20newsgroups

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.utils import shuffle

from sklearn.metrics.pairwise import cosine_similarity

## 1. Configuración del conjunto de búsqueda.

__*Para esta parte usaremos el conjunto de datos de 20Newsgroups, disponible en scikit-learn y que ya se ha usado en el notebook de ejemplo.*__

__*El objetivo de esta parte es poner realizar un buscador de mensajes en un foro, para lo cual sólo usaremos la parte de entrenamiento del conjunto de dato como los mensajes a recuperar por nuestro buscador.*__

In [2]:
# Estructura de datos de los mensajes del buscador.
## Devuelve un objeto "bunch" con distintos atributos.
searcherBunch = fetch_20newsgroups(
    subset = 'train',
    shuffle = True,
    random_state = 111
)

In [3]:
# data: array con los mensajes.
print('Nº mensajes:', len(searcherBunch.data))
print('---------------')
print('Mensaje con índice 10:', '\n')
print(searcherBunch.data[10])

Nº mensajes: 11314
---------------
Mensaje con índice 10: 

From: rgasch@nl.oracle.com (Robert Gasch)
Subject: Overriding Default Behaviour
Organization: Oracle Europe
Lines: 23
X-Newsreader: TIN [version 1.1 PL8]

Usually when I start up an application, I first get the window outline
on my display. I then have to click on the mouse button to actually
place the window on the screen. Yet when I specify the -geometry 
option the window appears right away, the properties specified by
the -geometry argument. The question now is:

How can I override the intermediary step of the user having to specify
window position with a mouseclick? I've tried explicitly setting window
size and position, but that did alter the normal program behaviour.

Thanks for any hints
---> Robert

PS: I'm working in plain X.



******************************************************************************
* Robert Gasch	       * Der erste Mai ist der Tag an dem die Stadt ins      *
* Oracle Engineering   * Freihe tr

In [4]:
# target: array con los índices de clase a los que corresponden los mensajes.
print('Nº mensajes:', len(searcherBunch.target))
print('---------------')
print('Índice de clase del mensaje 10:', searcherBunch.target[10])

Nº mensajes: 11314
---------------
Índice de clase del mensaje 10: 5


In [5]:
# target_names: array con todas las clases posibles.
print('Nº clases:', len(searcherBunch.target_names))
print('---------------')
print('Clase del mensaje 10:', searcherBunch.target_names[searcherBunch.target[10]])

Nº clases: 20
---------------
Clase del mensaje 10: comp.windows.x


## 2. Configuración de la vectorización.

__*Utilizaremos una representación de la bolsa de palabras con las siguientes opciones*__

- Bolsa no binaria: se cuenta la frecuencia de las palabras en cada mensaje.
- Diccionario: words.txt
- Palabras vacías: 'english'.
- Rango de n-gramas: (1,1).

In [6]:
# Convertimos el fichero del diccionario, 'words.txt', en una lista.
## Para conseguir un mejor rendimiento, transformamos las palabras del diccionario a minúscula,
## pues nuestro vectorizador hará lo propio al analizar los mensajes.
with open('words.txt') as f:
    words = np.unique(f.read().lower().splitlines())
    
## * Necesario eliminar duplicados, que reduce de forma despreciable el tamaño del diccionario.

In [7]:
print('Tamaño del diccionario:', len(words), 'palabras.')

Tamaño del diccionario: 466547 palabras.


In [8]:
# Vectorizador de la base de mensajes del buscador.
searcher_vectorizer = CountVectorizer(
    lowercase = True, # Por defecto es así.
    binary = False, # Se cuenta la frecuencia de aparición de las palabras.
    vocabulary = words, # Diccionario predeterminado. No se incluirán palabras nuevas.
    stop_words = 'english', # Palabras vacías que no se contarán.
    ngram_range = (1,1) # Sólo monogramas.
)

In [9]:
# Vectorizamos los datos del conjunto de mensajes del buscador.
## Como tenemos un vocabulario predeterminado, únicamente utilizamos la función 'transform'.
searcher_vd = searcher_vectorizer.transform(searcherBunch.data)

## 3. Análisis de mensajes del conjunto de prueba.

__*Tomar 3 mensajes del conjunto de prueba para cada clase, y utilizar cada uno de dichos mensajes como consulta para recuperar los mensajes del conjunto de entrenamiento que más se parezcan a la consulta. Utiliza los siguientes pasos:*__

1. Usar la distancia del coseno (cosine_similarity) entre el mensaje de la consulta y los mensajes de entrenamiento.
2. Ordenar los resultados de mayor a menor relevancia con la consulta.
3. Calcular la precisión de la lista de resultados con nivel de exhaustividad 3 y 10.
4. Calcular los valores de precisión media, para cada nivel de exhaustivida y para cada conjunto de datos.

In [10]:
# Obtenemos los mensajes del conjunto de prueba para extraer los mensajes sobre los que operaremos.
testBunch = fetch_20newsgroups(
    subset = 'test',
    shuffle = True,
    random_state = 111
)

### 3.1. Método de obtención de mensajes aleatorios.

In [11]:
## Podemos utilizar la función 'argwhere' de numpy para obtener un array con los índices de los
## elementos del array que verifican una determinada propiedad: en este caso, los elementos de
## la misma clase.
### Vemos un ejemplo con la clase 0.
ex_class0idx = np.argwhere(testBunch.target == 0)[:, 0]

## [:, 0] necesario porque devuelve ndarray: [[],[],...]

In [12]:
## En efecto, el array de índices obtenido hace referencia a índices de mensajes asociados 
## con ese tema.
print('Clase 0:', testBunch.target_names[0])
print('---------------')
print(testBunch.data[ex_class0idx[0]])

Clase 0: alt.atheism
---------------
From: decay@cbnewsj.cb.att.com (dean.kaflowitz)
Subject: Re: Origins of the bible.
Organization: AT&T
Distribution: na
Keywords: bible
Lines: 21

In article <1993Apr19.141112.15018@cs.nott.ac.uk>, eczcaw@mips.nott.ac.uk (A.Wainwright) writes:
> Hi,
> 
> I have been having an argument about the origins of the bible lately with
> a theist acquaintance.  He stated that thousands of bibles were discovered
> at a certain point in time which were syllable-perfect.  This therefore
> meant that there must have been one copy at a certain time; the time quoted
> by my acquaintace was approximately 50 years after the death of Jesus.

You can tell your friend from me that I was in a publisher's
warehouse one time and saw thousands of copies of The Joy of
Cooking and every one of them was syllable-perfect.

I have since sold all I own and become a follower of The Joy
of Cooking.  The incident I mentioned convinced me, once and
for all, that The Joy of Cooking is

In [13]:
## Podríamos usar los 3 primeros mensajes asociados a los tres primeros índices del array.
## No obstante, para darle un elemento de aleatoriedad, usaremos la función 'shuffle' de
## sklearn, para obtener 3 índices aleatorios.
ex_workIdx = shuffle(
    ex_class0idx,
    n_samples = 3,
    random_state = 111,
)

In [14]:
print('Índices obtenidos:')
print(ex_workIdx)

Índices obtenidos:
[3542 5499  335]


### 3.2. Obtención de 3 mensajes por clase.

In [15]:
# Obtenemos ahora los 3 mensajes para cada clase.
numClasses = len(testBunch.target_names) # Número de clases.

testQueries = list() # Mensajes de prueba.
testIndices = list() # Índices asociados.

for cat in range( numClasses ):
    ## Nueva lista.
    testQueries.append(list())
    testIndices.append(list())
    ## Índices asociados a la clase 'cat'.
    cat_idx = np.argwhere(testBunch.target == cat)[:, 0]
    ## Shuffle de los índices.
    cat_election = shuffle(
        cat_idx,
        n_samples = 3,
        random_state = 111
    )
    
    ## Guardamos los mensajes elegidos.
    for idx in cat_election:
        testIndices[cat].append(idx)
        testQueries[cat].append(testBunch.data[idx])

### 3.3. Cálculo de la similitud del coseno.

In [16]:
# Vamos a proceder a calcular la similaridad del coseno clase a clase con la bolsa 
# de palabras general.
## Para cada clase y mensaje de prueba, tenemos una lista ordenada por mayor similitud
## de valores (índice, valor).
searcherScores = list()

for cat in range(numClasses):
    # Nueva lista.
    searcherScores.append(list())
    # Vectorizamos los 3 mensajes de prueba de la clase.
    cat_testvd = searcher_vectorizer.transform(testQueries[cat])
    
    # Calculamos la similaridad del coseno.
    cat_distance = cosine_similarity(cat_testvd, searcher_vd)
    
    # Guardamos las listas de distancias, ordenadas por similitud de mayor a menor.
    for i in range(3):
        searcherScores[cat].append(list())
        searcherScores[cat][i] = sorted(
            list(enumerate(cat_distance[i])),
            key = lambda x: x[1],
            reverse = True
        )

### 3.4. Cálculo de la precisión para los niveles de exhaustividad 3 y 10.

In [17]:
# Cálculo de precisión con nivel de exhaustividad 'exh'
def getAccuracy(scoreList, cat, exh):
    if exh == 0:
        return 0
    
    relev = 0
    for (idx, _) in scoreList[:exh]:
        if searcherBunch.target[idx] == cat:
            relev += 1
            
    return relev / exh

In [18]:
accuracy_ex3 = list()

for cat in range(numClasses):
    # Nueva lista.
    accuracy_ex3.append(list())
    
    for i in range(3):
        accuracy_ex3[cat].append(list)
        accuracy_ex3[cat][i] = getAccuracy(searcherScores[cat][i], cat, 3)

In [19]:
# Precisión con exhaustividad 3 para cada clase y mensaje.
for cat in range(numClasses):
    print(searcherBunch.target_names[cat])
    print(accuracy_ex3[cat])

alt.atheism
[1.0, 0.3333333333333333, 1.0]
comp.graphics
[0.3333333333333333, 0.3333333333333333, 0.6666666666666666]
comp.os.ms-windows.misc
[0.0, 0.3333333333333333, 0.0]
comp.sys.ibm.pc.hardware
[0.0, 1.0, 0.0]
comp.sys.mac.hardware
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
comp.windows.x
[0.6666666666666666, 0.6666666666666666, 1.0]
misc.forsale
[0.3333333333333333, 1.0, 0.3333333333333333]
rec.autos
[0.6666666666666666, 1.0, 0.6666666666666666]
rec.motorcycles
[1.0, 0.3333333333333333, 1.0]
rec.sport.baseball
[0.3333333333333333, 0.0, 1.0]
rec.sport.hockey
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
sci.crypt
[1.0, 1.0, 0.6666666666666666]
sci.electronics
[1.0, 0.0, 0.0]
sci.med
[0.6666666666666666, 0.0, 0.0]
sci.space
[0.0, 0.0, 1.0]
soc.religion.christian
[0.6666666666666666, 0.0, 1.0]
talk.politics.guns
[0.3333333333333333, 1.0, 0.3333333333333333]
talk.politics.mideast
[0.0, 1.0, 1.0]
talk.politics.misc
[1.0, 0.0, 0.3333333333333333]
talk.re

In [20]:
accuracy_ex10 = list()

for cat in range(numClasses):
    # Nueva lista.
    accuracy_ex10.append(list())
    
    for i in range(3):
        accuracy_ex10[cat].append(list)
        accuracy_ex10[cat][i] = getAccuracy(searcherScores[cat][i], cat, 10)

In [21]:
# Precisión con exhaustividad 10 para cada clase y mensaje.
for cat in range(numClasses):
    print(searcherBunch.target_names[cat])
    print(accuracy_ex10[cat])

alt.atheism
[0.5, 0.4, 1.0]
comp.graphics
[0.2, 0.3, 0.4]
comp.os.ms-windows.misc
[0.0, 0.4, 0.1]
comp.sys.ibm.pc.hardware
[0.1, 1.0, 0.1]
comp.sys.mac.hardware
[0.5, 0.6, 0.4]
comp.windows.x
[0.9, 0.9, 1.0]
misc.forsale
[0.5, 0.6, 0.4]
rec.autos
[0.4, 0.9, 0.3]
rec.motorcycles
[1.0, 0.3, 1.0]
rec.sport.baseball
[0.3, 0.1, 0.8]
rec.sport.hockey
[0.4, 0.2, 0.1]
sci.crypt
[1.0, 0.5, 0.9]
sci.electronics
[0.8, 0.3, 0.0]
sci.med
[0.3, 0.0, 0.0]
sci.space
[0.0, 0.1, 1.0]
soc.religion.christian
[0.8, 0.0, 0.9]
talk.politics.guns
[0.4, 0.6, 0.2]
talk.politics.mideast
[0.0, 1.0, 1.0]
talk.politics.misc
[0.4, 0.0, 0.2]
talk.religion.misc
[0.6, 0.6, 0.1]


### 3.5. Cálculo de la precisión media para cada nivel de exhaustividad.

In [22]:
# Calculamos la precisión para todos los niveles de exhaustividad hasta un máximo.
maxExh = 20

accuracy_ex = list()

for exh in range(0, maxExh + 1):
    accuracy_ex.append(list())
    for cat in range(numClasses):
        accuracy_ex[exh].append(list())
        
        for i in range(3):
            accuracy_ex[exh][cat].append(list)
            accuracy_ex[exh][cat][i] = getAccuracy(searcherScores[cat][i], cat, exh)        

In [23]:
# Obtenemos las medias de los 3 mensajes de prueba, para cada clase y nivel de exhaustividad.
accMean_ex = list()

for exh in range(0, maxExh + 1):
    accMean_ex.append(list())
    for cat in range(numClasses):
        accMean_ex[exh].append(list())
        
        accMean_ex[exh][cat] = np.mean(accuracy_ex[exh][cat])

In [24]:
# Precisión con exhaustividad 10 para cada clase y grupo de mensajes.
for exh in range(maxExh + 1):
    print('Exhaustividad:', exh)
    print(accMean_ex[exh])

Exhaustividad: 0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Exhaustividad: 1
[1.0, 0.6666666666666666, 0.0, 0.3333333333333333, 0.3333333333333333, 1.0, 0.6666666666666666, 1.0, 1.0, 0.6666666666666666, 0.6666666666666666, 1.0, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.6666666666666666, 0.3333333333333333, 0.6666666666666666, 0.3333333333333333, 1.0]
Exhaustividad: 2
[0.8333333333333334, 0.6666666666666666, 0.16666666666666666, 0.3333333333333333, 0.5, 0.8333333333333334, 0.5, 1.0, 0.8333333333333334, 0.5, 0.3333333333333333, 1.0, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.5, 0.5, 0.6666666666666666, 0.3333333333333333, 0.6666666666666666]
Exhaustividad: 3
[0.7777777777777777, 0.4444444444444444, 0.1111111111111111, 0.3333333333333333, 0.3333333333333333, 0.7777777777777777, 0.5555555555555555, 0.7777777777777777, 0.7777777777777777, 0.4444444444444444, 0.3333333333333333, 0.888888888888888

### 3.6. Análisis de los resultados de precisión.

Para analizar los resultados de precisión de cada clase, vamos a comprobar las medias de precisión para los niveles de exhaustividad 5, 10, 15 y 20.

In [25]:
# Para nivel 5.
print('Nivel de exhaustividad: 5')
print('-------------')
for cat in range(numClasses):
    print(searcherBunch.target_names[cat], '->', accMean_ex[5][cat])

Nivel de exhaustividad: 5
-------------
alt.atheism -> 0.6666666666666666
comp.graphics -> 0.3333333333333333
comp.os.ms-windows.misc -> 0.19999999999999998
comp.sys.ibm.pc.hardware -> 0.39999999999999997
comp.sys.mac.hardware -> 0.4666666666666666
comp.windows.x -> 0.8666666666666667
misc.forsale -> 0.6
rec.autos -> 0.7333333333333334
rec.motorcycles -> 0.7333333333333334
rec.sport.baseball -> 0.4000000000000001
rec.sport.hockey -> 0.3333333333333333
sci.crypt -> 0.8666666666666667
sci.electronics -> 0.39999999999999997
sci.med -> 0.13333333333333333
sci.space -> 0.3333333333333333
soc.religion.christian -> 0.6
talk.politics.guns -> 0.5333333333333333
talk.politics.mideast -> 0.6666666666666666
talk.politics.misc -> 0.4000000000000001
talk.religion.misc -> 0.5333333333333333


In [26]:
# Para nivel 10.
print('Nivel de exhaustividad: 10')
print('-------------')
for cat in range(numClasses):
    print(searcherBunch.target_names[cat], '->', accMean_ex[10][cat])

Nivel de exhaustividad: 10
-------------
alt.atheism -> 0.6333333333333333
comp.graphics -> 0.3
comp.os.ms-windows.misc -> 0.16666666666666666
comp.sys.ibm.pc.hardware -> 0.4000000000000001
comp.sys.mac.hardware -> 0.5
comp.windows.x -> 0.9333333333333332
misc.forsale -> 0.5
rec.autos -> 0.5333333333333333
rec.motorcycles -> 0.7666666666666666
rec.sport.baseball -> 0.4000000000000001
rec.sport.hockey -> 0.23333333333333336
sci.crypt -> 0.7999999999999999
sci.electronics -> 0.3666666666666667
sci.med -> 0.09999999999999999
sci.space -> 0.3666666666666667
soc.religion.christian -> 0.5666666666666668
talk.politics.guns -> 0.39999999999999997
talk.politics.mideast -> 0.6666666666666666
talk.politics.misc -> 0.20000000000000004
talk.religion.misc -> 0.43333333333333335


In [27]:
# Para nivel 15.
print('Nivel de exhaustividad: 15')
print('-------------')
for cat in range(numClasses):
    print(searcherBunch.target_names[cat], '->', accMean_ex[15][cat])

Nivel de exhaustividad: 15
-------------
alt.atheism -> 0.5555555555555556
comp.graphics -> 0.2888888888888889
comp.os.ms-windows.misc -> 0.17777777777777778
comp.sys.ibm.pc.hardware -> 0.4222222222222222
comp.sys.mac.hardware -> 0.37777777777777777
comp.windows.x -> 0.8444444444444444
misc.forsale -> 0.4444444444444444
rec.autos -> 0.4888888888888889
rec.motorcycles -> 0.7999999999999999
rec.sport.baseball -> 0.37777777777777777
rec.sport.hockey -> 0.2222222222222222
sci.crypt -> 0.7777777777777777
sci.electronics -> 0.35555555555555557
sci.med -> 0.13333333333333333
sci.space -> 0.3333333333333333
soc.religion.christian -> 0.5777777777777778
talk.politics.guns -> 0.3111111111111111
talk.politics.mideast -> 0.6666666666666666
talk.politics.misc -> 0.19999999999999998
talk.religion.misc -> 0.39999999999999997


In [28]:
# Para nivel 20.
print('Nivel de exhaustividad: 20')
print('-------------')
for cat in range(numClasses):
    print(searcherBunch.target_names[cat], '->', accMean_ex[20][cat])

Nivel de exhaustividad: 20
-------------
alt.atheism -> 0.5499999999999999
comp.graphics -> 0.26666666666666666
comp.os.ms-windows.misc -> 0.18333333333333335
comp.sys.ibm.pc.hardware -> 0.39999999999999997
comp.sys.mac.hardware -> 0.35000000000000003
comp.windows.x -> 0.8666666666666667
misc.forsale -> 0.4333333333333333
rec.autos -> 0.48333333333333334
rec.motorcycles -> 0.7666666666666666
rec.sport.baseball -> 0.3833333333333333
rec.sport.hockey -> 0.26666666666666666
sci.crypt -> 0.7333333333333334
sci.electronics -> 0.31666666666666665
sci.med -> 0.10000000000000002
sci.space -> 0.3333333333333333
soc.religion.christian -> 0.5666666666666668
talk.politics.guns -> 0.3333333333333333
talk.politics.mideast -> 0.6666666666666666
talk.politics.misc -> 0.16666666666666666
talk.religion.misc -> 0.3333333333333333


In [29]:
accMean_ex[5]

[0.6666666666666666,
 0.3333333333333333,
 0.19999999999999998,
 0.39999999999999997,
 0.4666666666666666,
 0.8666666666666667,
 0.6,
 0.7333333333333334,
 0.7333333333333334,
 0.4000000000000001,
 0.3333333333333333,
 0.8666666666666667,
 0.39999999999999997,
 0.13333333333333333,
 0.3333333333333333,
 0.6,
 0.5333333333333333,
 0.6666666666666666,
 0.4000000000000001,
 0.5333333333333333]

In [30]:
precisionDF = pd.DataFrame(
    data = np.transpose([accMean_ex[5],accMean_ex[10],accMean_ex[15],accMean_ex[20]]),
    columns = ['% lvl 5','% lvl 10','% lvl 15','% lvl 20'],
    index = searcherBunch.target_names
    )

In [31]:
precisionDF * 100

,% lvl 5,% lvl 10,% lvl 15,% lvl 20
alt.atheism,66.666667,63.333333,55.555556,55.000000
comp.graphics,33.333333,30.000000,28.888889,26.666667
comp.os.ms-windows.misc,20.000000,16.666667,17.777778,18.333333
comp.sys.ibm.pc.hardware,40.000000,40.000000,42.222222,40.000000
comp.sys.mac.hardware,46.666667,50.000000,37.777778,35.000000
comp.windows.x,86.666667,93.333333,84.444444,86.666667
misc.forsale,60.000000,50.000000,44.444444,43.333333
rec.autos,73.333333,53.333333,48.888889,48.333333
rec.motorcycles,73.333333,76.666667,80.000000,76.666667
rec.sport.baseball,40.000000,40.000000,37.777778,38.333333


In [32]:
precisionDF.mean(axis = 1)

alt.atheism                 0.601389
comp.graphics               0.297222
comp.os.ms-windows.misc     0.181944
comp.sys.ibm.pc.hardware    0.405556
comp.sys.mac.hardware       0.423611
comp.windows.x              0.877778
misc.forsale                0.494444
rec.autos                   0.559722
rec.motorcycles             0.766667
rec.sport.baseball          0.390278
rec.sport.hockey            0.263889
sci.crypt                   0.794444
sci.electronics             0.359722
sci.med                     0.116667
sci.space                   0.341667
soc.religion.christian      0.577778
talk.politics.guns          0.394444
talk.politics.mideast       0.666667
talk.politics.misc          0.241667
talk.religion.misc          0.425000
dtype: float64

In [33]:
exhPrecision = precisionDF.mean(axis = 1).values

Como podemos observar en la tabla, las medias de precision son bastante variadas para cada clase en todos los niveles de exhaustividad mostrados. Así, hay clases con muy poca precisión, como 'comp.os.ms-windows.misc', con valores inferiores al 20 %, y otras clases con precisión muy alta, como 'comp.windows.x'.

Esta divergencia en las precisiones de cada clase está causada principalmente por la presencia de clases con temáticas muy parecidas, como las que hemos mencionado antes u otras como 'talk.religion.misc', 'soc.religion.christian' y 'alt.atheism'.

La clase con peores resultados de precisión ha sido 'sci.med', que sólo ha superado el 15 % en el nivel de exhaustividad 5. (índice: numClasses - 7)

In [34]:
lowPrec_idx = numClasses - 7
searcherBunch.target_names[lowPrec_idx]

'sci.med'

In [35]:
# Vemos el primer mensaje de consulta.
print('Clase:', testBunch.target_names[lowPrec_idx])
print('---------------')
print(testQueries[lowPrec_idx][0])

Clase: sci.med
---------------
From: kmldorf@utdallas.edu (George Kimeldorf)
Subject: Re: Opinions on Allergy (Hay Fever) shots?
Nntp-Posting-Host: heath.utdallas.edu
Organization: Univ. of Texas at Dallas
Lines: 20

In article <1993Apr29.173817.25867@nntpd2.cxo.dec.com> tung@paaiec.enet.dec.com () writes:
>
>I have just started taking allergy shots a month ago and is 
>still wondering what I am getting into. A friend of mine told
>me that the body change every 7 years (whatever that means)
>and I don't need those antibody-building allergy shots at all.
>Does that make sense to anyone?
>
>BTW, can someone summarize what is in the Consumer Report
>February, 1988 article?

I am reluctant to summarize it, for then you will have my opinion of what the
article says, rather than your own opinion.  I think it is important enough
for you to take the trouble to go to the library and get the article.  The
title is "The shot doctors" and it appears on Pages 96-100 of the February,
1988 issue of C

In [36]:
# Vemos la lista de las 10 primeros textos con más similitud.
searcherScores[lowPrec_idx][0][:10]

[(4928, 0.37935515451014556),
 (5511, 0.32318286529871065),
 (475, 0.28600869479648466),
 (7528, 0.2766534180712167),
 (5341, 0.2721238827974114),
 (1382, 0.2624997436527192),
 (5108, 0.26159300607748565),
 (9263, 0.26159300607748565),
 (10208, 0.25729584295741104),
 (813, 0.2568112826254347)]

In [37]:
# Vemos las clases de los primeros 10 mensajes.
for idx, _ in searcherScores[lowPrec_idx][0][:10]:
    print('Clase:', searcherBunch.target_names[searcherBunch.target[idx]])

Clase: sci.med
Clase: sci.med
Clase: talk.politics.misc
Clase: talk.politics.mideast
Clase: comp.sys.mac.hardware
Clase: rec.sport.baseball
Clase: rec.autos
Clase: sci.med
Clase: talk.politics.guns
Clase: comp.sys.ibm.pc.hardware


In [38]:
# En este caso hay 3/10 mensajes que coinciden con la clase del test.
## El porcentaje de media se ve reducido por la precisión de los otros 2 mensajes de prueba.
## Vemos el 3 texto encontrado y buscamos similitudes que hayan generado el 'error'.
lowPrec_checkidx = searcherScores[lowPrec_idx][0][2][0]
print( searcherBunch.data[lowPrec_checkidx] )

From: steveh@thor.isc-br.com (Steve Hendricks)
Subject: Re: Top Ten Reasons Not to Aid Russians
Summary: Constitutional Basis of Foreign Aid
Organization: Historical Accuracy, Inc.
Lines: 22
Nntp-Posting-Host: thor.isc-br.com

In article <C513wJ.75y@encore.com> rcollins@ns.encore.com (Roger Collins) writes:
>...
>I ask myself, what law could we pass to prevent government from doing
>stupid, frivilous things with OUR money?  Then I think, the Constitution
>was supposed to do that.  Could someone please tell me what legitimate
>constitutional power the federal government is using when it takes money
>from my paycheck and gives it to needy countries?  Seriously.
>
>Roger Collins
>

Since you asked, Article I Section 1.  Article I Section 8.  Article I 
Section 10.  Article II Section 2.  Article VI.  Sixteenth Amendment.

With this as a guide, try reading it yourself.

jsh

--
Steve Hendricks                        |  DOMAIN:  steveh@thor.ISC-BR.COM   
"One thing about data, it sure does 

In [39]:
# Para analizar mejor la similitud entre estos mensajes vemos el valor de las vectorizaciones.
## Aquí las del mensaje de prueba.
lowPrec_testvd = searcher_vectorizer.transform(testQueries[lowPrec_idx])
print(lowPrec_testvd[0])

  (0, 1725)	1
  (0, 7513)	1
  (0, 10862)	4
  (0, 12918)	1
  (0, 16800)	1
  (0, 18348)	1
  (0, 21406)	1
  (0, 25073)	6
  (0, 45931)	1
  (0, 52445)	1
  (0, 53046)	1
  (0, 65691)	1
  (0, 78260)	2
  (0, 82212)	2
  (0, 91167)	1
  (0, 94858)	1
  (0, 97062)	2
  (0, 111057)	1
  (0, 111238)	1
  (0, 111924)	1
  (0, 123627)	1
  (0, 124988)	1
  (0, 131163)	1
  (0, 136450)	2
  (0, 137850)	1
  :	:
  (0, 303588)	1
  (0, 309635)	1
  (0, 326574)	1
  (0, 332778)	1
  (0, 334230)	1
  (0, 334246)	1
  (0, 348504)	1
  (0, 357443)	1
  (0, 362266)	1
  (0, 362293)	5
  (0, 379250)	1
  (0, 385956)	1
  (0, 388849)	2
  (0, 395859)	1
  (0, 401968)	1
  (0, 403850)	1
  (0, 407658)	1
  (0, 408350)	2
  (0, 415699)	1
  (0, 417244)	1
  (0, 432220)	1
  (0, 445565)	1
  (0, 460170)	1
  (0, 462007)	1
  (0, 463615)	1


In [40]:
## Aquí las del mensaje obtenido.
print(searcher_vd[lowPrec_checkidx])

  (0, 2360)	1
  (0, 7978)	2
  (0, 12868)	1
  (0, 25073)	6
  (0, 25869)	1
  (0, 25878)	1
  (0, 35130)	1
  (0, 38197)	1
  (0, 48727)	4
  (0, 53387)	1
  (0, 77616)	2
  (0, 78260)	5
  (0, 82057)	1
  (0, 82058)	2
  (0, 87005)	1
  (0, 92745)	1
  (0, 95826)	1
  (0, 111238)	1
  (0, 111527)	1
  (0, 111770)	1
  (0, 122839)	2
  (0, 136514)	1
  (0, 144185)	1
  (0, 155618)	1
  (0, 159198)	2
  :	:
  (0, 310423)	1
  (0, 326621)	1
  (0, 327156)	1
  (0, 340845)	2
  (0, 344011)	1
  (0, 352952)	4
  (0, 358320)	1
  (0, 365830)	1
  (0, 380879)	2
  (0, 384355)	1
  (0, 385956)	1
  (0, 388854)	1
  (0, 391305)	1
  (0, 391644)	1
  (0, 395849)	1
  (0, 399521)	1
  (0, 403817)	1
  (0, 403835)	1
  (0, 403850)	1
  (0, 404269)	4
  (0, 416319)	1
  (0, 443467)	1
  (0, 443796)	1
  (0, 447517)	1
  (0, 462007)	1


In [41]:
# Definimos una función para encontrar los términos comunes.
def getCommonTerms(terms1, terms2):
    common = list()
    for term in terms1:
        if term in terms2:
            common.append(term)
            
    return common

In [42]:
lowPrec_common = getCommonTerms(lowPrec_testvd[0].indices, searcher_vd[lowPrec_checkidx].indices)

In [43]:
for term in lowPrec_common:
    print(searcher_vectorizer.get_feature_names()[term])

article
com
does
host
lines
nntp
organization
posting
subject
think
writes


Como podemos comprobar, los cálculos de similaridad se han hecho con palabras que neutras que no guardan relación temática con ninguna de las dos clases. No obstante, el valor de similaridad ha debido de ser suficientemente alto como para que se halla encontrado el texto en tercera posición.

Podemos encontrar justificación en este error en que se han hecho los cálculos con palabras que con toda probabilidad tendrán presencia en una gran cantidad de textos. Sin embargo, en los cálculos sólo se ha tenido en cuenta la frecuencia de aparición en cada documento. Es por esto que una posible solución a esta baja precisión de clasificación sea transformar los datos vectorizados con TF/IDF.

##  4. Configuración de la vectorización TF/IDF.

In [44]:
# Obtenemos un transformador TF-IDF.
tfidfer = TfidfTransformer()

In [45]:
# Transformamos la vectorización del conjunto de búsqueda con TF/IDF.
searcher_pp = tfidfer.fit_transform(searcher_vd)

In [46]:
## Nos quedan los datos de la bolsa de palabras transformados por TF-IDF.
print(searcher_pp[:3])

  (0, 432272)	0.053203153127324175
  (0, 401139)	0.171475198623231
  (0, 385956)	0.02748202571622332
  (0, 348470)	0.0794130483498687
  (0, 340313)	0.4361941232428854
  (0, 317626)	0.1491891935671113
  (0, 303588)	0.04945228420082254
  (0, 290427)	0.12752078811343487
  (0, 290418)	0.16461950886364463
  (0, 285785)	0.13314895342831146
  (0, 280326)	0.1719370922810174
  (0, 269741)	0.028589732391019626
  (0, 253348)	0.05117447878622388
  (0, 252059)	0.24589790389074564
  (0, 220041)	0.09539359172260418
  (0, 216990)	0.19665672408013363
  (0, 216701)	0.02757203907085047
  (0, 184085)	0.15312189463205753
  (0, 177951)	0.050814484529593966
  (0, 167157)	0.12993728462729345
  (0, 146790)	0.1692754628725507
  (0, 136842)	0.15456865106950193
  (0, 135069)	0.20779972660819343
  (0, 134099)	0.13908337170321478
  (0, 123759)	0.18476074223883787
  :	:
  (2, 86096)	0.039490563846254476
  (2, 83357)	0.12895072989315645
  (2, 82775)	0.04796790921152019
  (2, 79441)	0.03960183978244082
  (2, 79400)	0.

## 5. Análisis de mensajes del conjunto de prueba con TF/IDF.

Puesto que ya tenemos los mensajes del conjunto de prueba elegidos y estos son independientes de la transformación TF/IDF, los mantenemos (testQueries, testIndices).

Pasamos directamente a calcular la similaridad del coseno.

### 5.1. Cálculo de la similitud del coseno.

In [47]:
# Vamos a proceder a calcular la similaridad del coseno clase a clase con la bolsa 
# de palabras general.
## Para cada clase y mensaje de prueba, tenemos una lista ordenada por mayor similitud
## de valores (índice, valor).
tfidf_searcherScores = list()

for cat in range(numClasses):
    # Nueva lista.
    tfidf_searcherScores.append(list())
    # Vectorizamos los 3 mensajes de prueba de la clase.
    cat_testvd = searcher_vectorizer.transform(testQueries[cat])
    # Transformamos los datos con TF/IDF.
    cat_testpp = tfidfer.transform(cat_testvd)
    
    # Calculamos la similaridad del coseno.
    cat_distance = cosine_similarity(cat_testpp, searcher_pp)
    
    # Guardamos las listas de distancias, ordenadas por similitud de mayor a menor.
    for i in range(3):
        tfidf_searcherScores[cat].append(list())
        tfidf_searcherScores[cat][i] = sorted(
            list(enumerate(cat_distance[i])),
            key = lambda x: x[1],
            reverse = True
        )

### 5.2. Cálculo de la precisión media para cada nivel de exhaustividad.

In [48]:
# Calculamos la precisión para todos los niveles de exhaustividad hasta un máximo.
maxExh = 20

tfidf_accuracy_ex = list()

for exh in range(0, maxExh + 1):
    tfidf_accuracy_ex.append(list())
    for cat in range(numClasses):
        tfidf_accuracy_ex[exh].append(list())
        
        for i in range(3):
            tfidf_accuracy_ex[exh][cat].append(list)
            tfidf_accuracy_ex[exh][cat][i] = getAccuracy(tfidf_searcherScores[cat][i], cat, exh)     

In [49]:
# Obtenemos las medias de los 3 mensajes de prueba, para cada clase y nivel de exhaustividad.
tfidf_accMean_ex = list()

for exh in range(0, maxExh + 1):
    tfidf_accMean_ex.append(list())
    for cat in range(numClasses):
        tfidf_accMean_ex[exh].append(list())
        
        tfidf_accMean_ex[exh][cat] = np.mean(tfidf_accuracy_ex[exh][cat])

In [50]:
# Precisión con exhaustividad 10 para cada clase y mensaje.
for exh in range(maxExh + 1):
    print('Exhaustividad:', exh)
    print(tfidf_accMean_ex[exh])

Exhaustividad: 0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Exhaustividad: 1
[0.6666666666666666, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 1.0, 1.0, 0.6666666666666666, 0.6666666666666666, 1.0, 0.3333333333333333, 0.6666666666666666, 0.3333333333333333, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.3333333333333333, 0.6666666666666666]
Exhaustividad: 2
[0.6666666666666666, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.5, 0.8333333333333334, 0.3333333333333333, 1.0, 0.8333333333333334, 0.5, 0.6666666666666666, 1.0, 0.3333333333333333, 0.6666666666666666, 0.3333333333333333, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.16666666666666666, 0.5]
Exhaustividad: 3
[0.6666666666666666, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.6666666666666666, 0.8888888888888888, 0.2222222222222222, 0.88888

### 5.3. Análisis de los resultados de precisión.

Procedemos de forma análoga al apartado anterior.

In [51]:
# Para nivel 5.
print('Nivel de exhaustividad: 5')
print('-------------')
for cat in range(numClasses):
    print(searcherBunch.target_names[cat], '->', tfidf_accMean_ex[5][cat])

Nivel de exhaustividad: 5
-------------
alt.atheism -> 0.7333333333333334
comp.graphics -> 0.4000000000000001
comp.os.ms-windows.misc -> 0.39999999999999997
comp.sys.ibm.pc.hardware -> 0.4666666666666666
comp.sys.mac.hardware -> 0.6
comp.windows.x -> 0.9333333333333332
misc.forsale -> 0.26666666666666666
rec.autos -> 0.8666666666666667
rec.motorcycles -> 0.7999999999999999
rec.sport.baseball -> 0.4666666666666666
rec.sport.hockey -> 0.46666666666666673
sci.crypt -> 0.8666666666666667
sci.electronics -> 0.4666666666666666
sci.med -> 0.4666666666666666
sci.space -> 0.3333333333333333
soc.religion.christian -> 0.7333333333333334
talk.politics.guns -> 0.4666666666666666
talk.politics.mideast -> 0.6666666666666666
talk.politics.misc -> 0.13333333333333333
talk.religion.misc -> 0.46666666666666673


In [52]:
# Para nivel 10.
print('Nivel de exhaustividad: 10')
print('-------------')
for cat in range(numClasses):
    print(searcherBunch.target_names[cat], '->', tfidf_accMean_ex[10][cat])

Nivel de exhaustividad: 10
-------------
alt.atheism -> 0.6
comp.graphics -> 0.39999999999999997
comp.os.ms-windows.misc -> 0.3
comp.sys.ibm.pc.hardware -> 0.3666666666666667
comp.sys.mac.hardware -> 0.3666666666666667
comp.windows.x -> 0.9666666666666667
misc.forsale -> 0.26666666666666666
rec.autos -> 0.7666666666666666
rec.motorcycles -> 0.8333333333333334
rec.sport.baseball -> 0.4666666666666666
rec.sport.hockey -> 0.4000000000000001
sci.crypt -> 0.8333333333333334
sci.electronics -> 0.43333333333333335
sci.med -> 0.43333333333333335
sci.space -> 0.3333333333333333
soc.religion.christian -> 0.6666666666666666
talk.politics.guns -> 0.39999999999999997
talk.politics.mideast -> 0.6666666666666666
talk.politics.misc -> 0.13333333333333333
talk.religion.misc -> 0.4000000000000001


In [53]:
# Para nivel 15.
print('Nivel de exhaustividad: 15')
print('-------------')
for cat in range(numClasses):
    print(searcherBunch.target_names[cat], '->', tfidf_accMean_ex[15][cat])

Nivel de exhaustividad: 15
-------------
alt.atheism -> 0.5555555555555555
comp.graphics -> 0.37777777777777777
comp.os.ms-windows.misc -> 0.2888888888888889
comp.sys.ibm.pc.hardware -> 0.37777777777777777
comp.sys.mac.hardware -> 0.37777777777777777
comp.windows.x -> 0.9333333333333332
misc.forsale -> 0.3333333333333333
rec.autos -> 0.6888888888888888
rec.motorcycles -> 0.8444444444444444
rec.sport.baseball -> 0.37777777777777777
rec.sport.hockey -> 0.37777777777777777
sci.crypt -> 0.8000000000000002
sci.electronics -> 0.39999999999999997
sci.med -> 0.35555555555555557
sci.space -> 0.35555555555555557
soc.religion.christian -> 0.6444444444444445
talk.politics.guns -> 0.39999999999999997
talk.politics.mideast -> 0.6666666666666666
talk.politics.misc -> 0.13333333333333333
talk.religion.misc -> 0.3333333333333333


In [54]:
# Para nivel 20.
print('Nivel de exhaustividad: 20')
print('-------------')
for cat in range(numClasses):
    print(searcherBunch.target_names[cat], '->', tfidf_accMean_ex[20][cat])

Nivel de exhaustividad: 20
-------------
alt.atheism -> 0.5833333333333334
comp.graphics -> 0.3833333333333333
comp.os.ms-windows.misc -> 0.26666666666666666
comp.sys.ibm.pc.hardware -> 0.3833333333333333
comp.sys.mac.hardware -> 0.39999999999999997
comp.windows.x -> 0.9333333333333332
misc.forsale -> 0.39999999999999997
rec.autos -> 0.65
rec.motorcycles -> 0.7999999999999999
rec.sport.baseball -> 0.4666666666666666
rec.sport.hockey -> 0.3833333333333333
sci.crypt -> 0.7666666666666666
sci.electronics -> 0.3666666666666667
sci.med -> 0.3
sci.space -> 0.3833333333333333
soc.religion.christian -> 0.6333333333333333
talk.politics.guns -> 0.4166666666666667
talk.politics.mideast -> 0.6666666666666666
talk.politics.misc -> 0.15
talk.religion.misc -> 0.25


In [55]:
tfidf_precisionDF = pd.DataFrame(
    data = np.transpose([tfidf_accMean_ex[5],tfidf_accMean_ex[10],tfidf_accMean_ex[15],tfidf_accMean_ex[20]]),
    columns = ['% lvl 5','% lvl 10','% lvl 15','% lvl 20'],
    index = searcherBunch.target_names
    )

In [56]:
tfidf_precisionDF * 100

,% lvl 5,% lvl 10,% lvl 15,% lvl 20
alt.atheism,73.333333,60.000000,55.555556,58.333333
comp.graphics,40.000000,40.000000,37.777778,38.333333
comp.os.ms-windows.misc,40.000000,30.000000,28.888889,26.666667
comp.sys.ibm.pc.hardware,46.666667,36.666667,37.777778,38.333333
comp.sys.mac.hardware,60.000000,36.666667,37.777778,40.000000
comp.windows.x,93.333333,96.666667,93.333333,93.333333
misc.forsale,26.666667,26.666667,33.333333,40.000000
rec.autos,86.666667,76.666667,68.888889,65.000000
rec.motorcycles,80.000000,83.333333,84.444444,80.000000
rec.sport.baseball,46.666667,46.666667,37.777778,46.666667


In [57]:
tfidf_precisionDF.mean(axis = 1)

alt.atheism                 0.618056
comp.graphics               0.390278
comp.os.ms-windows.misc     0.313889
comp.sys.ibm.pc.hardware    0.398611
comp.sys.mac.hardware       0.436111
comp.windows.x              0.941667
misc.forsale                0.316667
rec.autos                   0.743056
rec.motorcycles             0.819444
rec.sport.baseball          0.444444
rec.sport.hockey            0.406944
sci.crypt                   0.816667
sci.electronics             0.416667
sci.med                     0.388889
sci.space                   0.351389
soc.religion.christian      0.669444
talk.politics.guns          0.420833
talk.politics.mideast       0.666667
talk.politics.misc          0.137500
talk.religion.misc          0.362500
dtype: float64

In [58]:
tfidf_exhPrecision = tfidf_precisionDF.mean(axis = 1).values

In [59]:
precisionDiffDF = pd.DataFrame(
    data = np.transpose([exhPrecision, tfidf_exhPrecision, tfidf_exhPrecision - exhPrecision]),
    index = searcherBunch.target_names,
    columns = ['Mean Exh Precision', 'TF/IDF Mean Exh Precision', 'Diff'])

In [60]:
precisionDiffDF * 100

,Mean Exh Precision,TF/IDF Mean Exh Precision,Diff
alt.atheism,60.138889,61.805556,1.666667
comp.graphics,29.722222,39.027778,9.305556
comp.os.ms-windows.misc,18.194444,31.388889,13.194444
comp.sys.ibm.pc.hardware,40.555556,39.861111,-0.694444
comp.sys.mac.hardware,42.361111,43.611111,1.250000
comp.windows.x,87.777778,94.166667,6.388889
misc.forsale,49.444444,31.666667,-17.777778
rec.autos,55.972222,74.305556,18.333333
rec.motorcycles,76.666667,81.944444,5.277778
rec.sport.baseball,39.027778,44.444444,5.416667


Como podemos ver en este DataFrame de comparación, ha habido cambios en la precisión, que principalemnte ha aumentado a nivel general (media de los niveles de exhaustividad 5,10,15 y 20) en todas las clases.

En concreto, la clase 'sci.med' que sin TF/IDF era la que obtenía una peor precisión ha aumentado considerablemente, aumentando en 27.2 puntos porcentuales, triplicando su rendimiento anterior.

No obstante, algunas clases han perdido precisión, como es 'misc.forsale', 'talk.politics.misc' o 'talk.religion.misc'. Todas estas clases contienen 'misc' que podemos intuir se refiere a 'miscelaneous', con lo que por la propia definición de la palabra, abarcan un amplio abanico de temáticas distintas. Por ello, es normal que su precisión baje.

Para comprobar la mejora con TF/IDF, vamos a volver a analizar el mismo mensaje del apartado anterior de la clase 'sci.med'.

In [61]:
# Vemos el primer mensaje de consulta.
print('Clase:', testBunch.target_names[lowPrec_idx])
print('---------------')
print(testQueries[lowPrec_idx][0])

Clase: sci.med
---------------
From: kmldorf@utdallas.edu (George Kimeldorf)
Subject: Re: Opinions on Allergy (Hay Fever) shots?
Nntp-Posting-Host: heath.utdallas.edu
Organization: Univ. of Texas at Dallas
Lines: 20

In article <1993Apr29.173817.25867@nntpd2.cxo.dec.com> tung@paaiec.enet.dec.com () writes:
>
>I have just started taking allergy shots a month ago and is 
>still wondering what I am getting into. A friend of mine told
>me that the body change every 7 years (whatever that means)
>and I don't need those antibody-building allergy shots at all.
>Does that make sense to anyone?
>
>BTW, can someone summarize what is in the Consumer Report
>February, 1988 article?

I am reluctant to summarize it, for then you will have my opinion of what the
article says, rather than your own opinion.  I think it is important enough
for you to take the trouble to go to the library and get the article.  The
title is "The shot doctors" and it appears on Pages 96-100 of the February,
1988 issue of C

In [62]:
# Vemos la lista de las 10 primeros textos con más similitud (TF/IDF)
tfidf_searcherScores[lowPrec_idx][0][:10]

[(4928, 0.37848615404995184),
 (9263, 0.30895960625777613),
 (10759, 0.20395767442306895),
 (3095, 0.19983538898830686),
 (6594, 0.16112015227772564),
 (8280, 0.15425621057648373),
 (606, 0.11762157219916747),
 (5710, 0.11708847053507439),
 (6717, 0.10753832215617506),
 (5207, 0.10621873684164523)]

In [63]:
# Vemos las clases de los primeros 10 mensajes.
for idx, _ in tfidf_searcherScores[lowPrec_idx][0][:10]:
    print('Clase:', searcherBunch.target_names[searcherBunch.target[idx]])

Clase: sci.med
Clase: sci.med
Clase: rec.sport.hockey
Clase: rec.sport.hockey
Clase: sci.med
Clase: rec.sport.hockey
Clase: sci.med
Clase: rec.sport.hockey
Clase: rec.sport.hockey
Clase: misc.forsale


In [64]:
# En este caso hay 4/10 mensajes que coinciden con la clase 'sci.med'.
## A continuación, vemos el tercer mensaje para comprobar por qué razón ha salido en esa posición.

In [65]:
lowPrec_tfidf_checkidx = tfidf_searcherScores[lowPrec_idx][0][2][0]
print( searcherBunch.data[lowPrec_tfidf_checkidx] )

From: bks2@cbnewsi.cb.att.com (bryan.k.strouse)
Subject: NHL RESULTS FOR GAMES PLAYED 4-15-93
Organization: AT&T
Keywords: thursday night's boxscores
Lines: 227



NHL RESULTS FOR GAMES PLAYED 4/15/93.

--------------------------------------------------------------------------------
                                  STANDINGS
      PATRICK              ADAMS              NORRIS              SMYTHE
 TM    W  L  T  PT   TM    W  L  T  PT   TM    W  L  T  PT   TM    W  L  T  PT
 
xPIT  56 21  7 119  xBOS  51 26  7 109  xCHI  47 25 12 106  xVAN  46 29  9 101
yWAS  42 34  7  91  yQUE  47 27 10 104  yDET  47 28  9 103  yCAL  43 30 11  97
yNJ   40 36  7  87  yMON  48 30  6 102  yTOR  44 29 11  99  yLA   39 35 10  88
yNYI  39 37  7  85  yBUF  38 36 10  86  ySTL  37 36 11  85  yWIN  40 37  7  87
 PHL  35 37 11  81   HAR  26 51  6  58   MIN  36 38 10  82   EDM  26 50  8  60
 NYR  34 38 11  79   OTT  10 70  4  24   TB   23 54  7  53   SJ   11 71  2  24

x - Clinched Division Title
y - Clinched Pl

In [66]:
# Para analizar mejor la similitud entre estos mensajes vemos el valor de las vectorizaciones.
## Aquí las del mensaje de prueba.
tfidf_lowPrec_testvd = searcher_vectorizer.transform(testQueries[lowPrec_idx])
tfidf_lowPrec_testpp = tfidfer.transform(tfidf_lowPrec_testvd)
print(tfidf_lowPrec_testpp[0])

  (0, 463615)	0.04724319965229287
  (0, 462007)	0.025024425282567573
  (0, 460170)	0.0692224538788182
  (0, 445565)	0.043209256781792396
  (0, 432220)	0.06543723080923797
  (0, 417244)	0.12323022369966806
  (0, 415699)	0.07172550845845237
  (0, 408350)	0.1225190560360241
  (0, 407658)	0.07594152074983447
  (0, 403850)	0.036221127243637324
  (0, 401968)	0.06707942942851652
  (0, 395859)	0.06619480369606806
  (0, 388849)	0.20239295787040046
  (0, 385956)	0.01492895686056773
  (0, 379250)	0.06392273865029698
  (0, 362293)	0.4406332196123408
  (0, 362266)	0.07444737911458087
  (0, 357443)	0.0636788529510171
  (0, 348504)	0.053296370759705086
  (0, 334246)	0.0736851786721945
  (0, 334230)	0.07160749237461562
  (0, 332778)	0.1119771969792787
  (0, 326574)	0.04890706293556722
  (0, 309635)	0.06892485684451462
  (0, 303588)	0.02686377725986899
  :	:
  (0, 137850)	0.10746445888331288
  (0, 136450)	0.18044406572111785
  (0, 131163)	0.11989892324838433
  (0, 124988)	0.13787298130546302
  (0, 1236

In [67]:
## Aquí las del mensaje obtenido.
print(searcher_pp[lowPrec_tfidf_checkidx])

  (0, 463506)	0.024984353739746813
  (0, 460236)	0.03768647622176421
  (0, 458879)	0.03609412370619475
  (0, 458803)	0.06811657599363809
  (0, 458342)	0.038824615551836125
  (0, 457229)	0.02441717450690523
  (0, 451919)	0.0201700831076684
  (0, 446929)	0.022011878855855364
  (0, 444702)	0.017127132355532076
  (0, 444672)	0.13566101219034285
  (0, 409637)	0.027742479336706396
  (0, 409481)	0.04150105668825779
  (0, 407826)	0.06649120472850208
  (0, 407658)	0.015824577623750714
  (0, 405658)	0.019358996782354258
  (0, 398236)	0.02091429632032012
  (0, 396420)	0.04235468725409338
  (0, 392215)	0.04883434901381046
  (0, 385956)	0.0031108731343412657
  (0, 381612)	0.11005939427927681
  (0, 380710)	0.023522232076483362
  (0, 379937)	0.025312116937327094
  (0, 379235)	0.07245876877386204
  (0, 378833)	0.02091429632032012
  (0, 378161)	0.02750249242998051
  :	:
  (0, 60450)	0.02567852501863169
  (0, 56679)	0.03680190262669202
  (0, 56271)	0.13101372245387174
  (0, 54192)	0.024687855861964307
 

In [68]:
# Obtenemos los términos comunes.
tfidf_lowPrec_common = getCommonTerms(tfidf_lowPrec_testpp[0].indices, searcher_pp[lowPrec_tfidf_checkidx].indices)

In [69]:
for term in tfidf_lowPrec_common:
    print(
        searcher_vectorizer.get_feature_names()[term], '->',
        tfidf_lowPrec_testpp[0, term], ':',
        searcher_pp[lowPrec_tfidf_checkidx, term]
        )

title -> 0.07594152074983447 : 0.015824577623750714
subject -> 0.01492895686056773 : 0.0031108731343412657
shots -> 0.4406332196123408 : 0.4590923725591761
organization -> 0.0155306921668714 : 0.0032362618145985625
lines -> 0.01497785447466575 : 0.0031210623441736488
com -> 0.05551750981225888 : 0.00578432677425246


Esta vez, como podemos observar por los valores de TF/IDF, las palabras poco relevantes: subject, title, organization, lines y com; tienen un valor asociado muy pequeño (< 0.1) por lo que su aportación a la similaridad del coseno será muy baja.

No obstante, sí hay una palabra con un peso más importante en ambos textos: *shots*. Es obvio, que los mensajes no tienen nada que ver, y que la similitud ha sido producida por la polisemia de la palabra 'shots' en inglés: mientras que en el mensaje de consulta se refiere a 'allergy shots', que son inyecciones para tratar enfermedades alérgicas; el mensaje obtenido se refiere a 'tiros' en un determinado deporte, pues el texto relata las estadísticas de los partidos de una liga deportiva.

Por tanto, como hemos podido ver, aunque se siguen produciendo fallos en la precisión, la aplicación del TF/IDF ha ayudado a establecer separaciones más nítidas entre textos, consiguiendo resultados más precisos con las búsquedas.

__*FIN*__